<a href="https://colab.research.google.com/github/wlvvs/FY22-Q2-AI-Hackathon-Anomaly-Detection/blob/master/notebooks/AnomalyDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [ ]:
url ='https://raw.githubusercontent.com/wlvvs/FY22-Q2-AI-Hackathon-Anomaly-Detection/master/data/raw/ABCDPublishingStatistics2017-2022_Master.csv'

In [ ]:
df = pd.read_csv(url,sep='|')

In [ ]:
df.shape

(11719, 27)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11719 entries, 0 to 11718
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   PKG_ID              11719 non-null  object
 1   PKG_JURIS           11719 non-null  object
 2   PKG_STATUS          11719 non-null  object
 3   PKG_TITLE           11719 non-null  object
 4   PKG_ISSUE_DATE      11719 non-null  object
 5   PKG_EFFECTIVE_DATE  11719 non-null  object
 6   PKG_APPROVAL_DATE   11719 non-null  object
 7   PKG_SUBMIT_DATE     11245 non-null  object
 8   PKG_RDY_DATE        11245 non-null  object
 9   PKG_FILED_DATE      11719 non-null  object
 10  PKG_PRINT_REQ_FLAG  2173 non-null   object
 11  PKG_DELETE_FLAG     11719 non-null  object
 12  PKG_FILING_NO       11719 non-null  object
 13  PKG_CREATE_DATE     11719 non-null  object
 14  PKG_ORG             11719 non-null  int64 
 15  PKG_RET_WHOL        11719 non-null  int64 
 16  PKG_PROJ_TYPE       11

In [ ]:
df.sample(5)

,PKG_ID,PKG_JURIS,PKG_STATUS,PKG_TITLE,PKG_ISSUE_DATE,PKG_EFFECTIVE_DATE,PKG_APPROVAL_DATE,PKG_SUBMIT_DATE,PKG_RDY_DATE,PKG_FILED_DATE,PKG_PRINT_REQ_FLAG,PKG_DELETE_FLAG,PKG_FILING_NO,PKG_CREATE_DATE,PKG_ORG,PKG_RET_WHOL,PKG_PROJ_TYPE,PKG_FILING_TYPE,PKG_BUS_UNIT,PKG_ADVICE_LETTER,PKG_STATE_SPECF1,PKG_STATE_SPECF2,PKG_STATE_SPECF3,PKG_STATE_SPECF4,PKG_PUBLISHED_DATE,PKG_VISIBLE,PKG_AUTO_PROCESS
2923,MU-18-0005,MICHRC,Approved,ILEC - Business Usage Rate Change,04/01/2018 00:00,04/01/2018 00:00,05/01/2018 08:07,31/10/2017 11:45,31/10/2017 12:31,08/11/2017 10:22,NaN,Y,MU-18-0005,31/10/2017 11:44,16,1,12,10,1,0,NaN,0,NaN,NaN,05/01/2018 08:48,Y,N
6596,IW-19-0032,ILTAR22,Approved,BLC AFL Back Up Bundle Promo III,21/06/2019 00:00,01/07/2019 00:00,01/07/2019 07:00,22/05/2019 11:53,21/06/2019 09:45,21/06/2019 15:08,NaN,Y,IW-19-0032,21/05/2019 15:43,16,2,2,9,33,0,NaN,NaN,NaN,NaN,01/07/2019 12:04,Y,N
5682,NC-19-0010,NORTH CAROLINA,Approved,Centrex Feature Package Increase,01/02/2019 00:00,01/02/2019 00:00,01/02/2019 11:42,18/12/2018 09:15,02/01/2019 12:45,21/01/2019 10:32,D,Y,NC-19-0010,11/12/2018 10:13,1,1,12,10,33,0,NaN,NaN,NaN,NaN,01/02/2019 12:20,Y,Z
5330,OH-18-0009,OHIO,Approved,Business Signature BOT Correction,12/12/2018 00:00,12/12/2018 00:00,12/12/2018 06:30,04/12/2018 11:15,04/12/2018 14:15,04/12/2018 15:06,NaN,Y,OH-18-0009,04/12/2018 11:06,8192,1,12,25,33,0,NaN,NaN,201885,NaN,12/12/2018 07:33,Y,N
11214,TX-21-0012,TEXAS,Approved,Withdraw AT&T CustomNet Sarvice and AT&T PRO W...,31/10/2021 00:00,31/10/2021 00:00,31/10/2021 07:30,22/09/2021 19:28,23/09/2021 16:36,25/10/2021 15:24,NaN,Y,TX-21-0012,22/09/2021 14:36,512,1,17,16,15,0,Y~,REM,202145,ALD,02/02/2022 11:12,Y,N


In [ ]:
df.describe()

,PKG_ORG,PKG_RET_WHOL,PKG_PROJ_TYPE,PKG_FILING_TYPE,PKG_BUS_UNIT,PKG_ADVICE_LETTER
count,11719.000000,11719.000000,11719.000000,11719.000000,11719.000000,1.171900e+04
mean,1278.167079,1.066217,10.735984,9.461046,14.348750,1.165211e+03
std,2745.948487,0.379386,5.161309,4.578924,12.227133,3.304283e+04
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,2.000000,1.000000,12.000000,9.000000,2.000000,0.000000e+00
50%,16.000000,1.000000,12.000000,10.000000,15.000000,0.000000e+00
75%,512.000000,1.000000,12.000000,10.000000,33.000000,0.000000e+00
max,8192.000000,8.000000,22.000000,26.000000,34.000000,2.017107e+06


In [ ]:
df.isnull().sum()

PKG_ID                   0
PKG_JURIS                0
PKG_STATUS               0
PKG_TITLE                0
PKG_ISSUE_DATE           0
PKG_EFFECTIVE_DATE       0
PKG_APPROVAL_DATE        0
PKG_SUBMIT_DATE        474
PKG_RDY_DATE           474
PKG_FILED_DATE           0
PKG_PRINT_REQ_FLAG    9546
PKG_DELETE_FLAG          0
PKG_FILING_NO            0
PKG_CREATE_DATE          0
PKG_ORG                  0
PKG_RET_WHOL             0
PKG_PROJ_TYPE            0
PKG_FILING_TYPE          0
PKG_BUS_UNIT             0
PKG_ADVICE_LETTER        0
PKG_STATE_SPECF1      7683
PKG_STATE_SPECF2      8087
PKG_STATE_SPECF3      6796
PKG_STATE_SPECF4      8337
PKG_PUBLISHED_DATE      50
PKG_VISIBLE              0
PKG_AUTO_PROCESS         0
dtype: int64

In [ ]:
df.columns

Index(['PKG_ID', 'PKG_JURIS', 'PKG_APPROVAL_DATE', 'PKG_SUBMIT_DATE',
       'PKG_RDY_DATE', 'PKG_FILED_DATE', 'PKG_DESCRIPTION', 'PKG_FILING_NO',
       'PKG_CREATE_DATE', 'PKG_ORG', 'PKG_RET_WHOL', 'PKG_PROJ_TYPE',
       'PKG_FILING_TYPE', 'PKG_BUS_UNIT', 'PKG_ADVICE_LETTER',
       'PKG_STATE_SPECF1', 'PKG_STATE_SPECF2', 'PKG_STATE_SPECF3',
       'PKG_STATE_SPECF4', 'PKG_PUBLISHED_DATE', 'PKG_AUTO_PROCESS',
       '# of Days'],
      dtype='object')

In [ ]:
df.values

array([['AL-17-0002', 'ALABAMA', '1-Jan-17', ..., '1-Jan-17', 'Z', 0],
       ['LA-17-0001', 'LOUISIANA', '1-Jan-17', ..., '1-Jan-17', 'Z', 0],
       ['LA-17-0002', 'LOUISIANA', '1-Jan-17', ..., '1-Jan-17', 'Z', 0],
       ...,
       ['TX-22-0013', 'TEXAS', '14-feb-22', ..., '15-feb-22', 'N', 1],
       ['WG-22-0008', 'WISCONSIN', '15-feb-22', ..., '15-feb-22', 'N', 0],
       ['SC-22-0008', 'SOUTH CAROLINA', '15-feb-22', ..., '15-feb-22',
        'Z', 0]], dtype=object)

In [ ]:
df.values.reshape(22,11719)

array([['AL-17-0002', 'ALABAMA', '1-Jan-17', ..., 9, 1, 0],
       [nan, nan, nan, ..., '31-mar-17',
        'With this filing the Company is removing PrimeOne and PrimePlus Volume Discounts for new commitments.',
        'NY-17-0506'],
       ['29-mar-17', 256, 1, ..., 'N', 0, 'MI-17-0011'],
       ...,
       [0, 'MO-21-0020', 'MISSOURI', ..., 17, 10, 16],
       [0, 'Y~', 'RPC', ..., '24-sep-21', '29-oct-21',
        'Withdraw AT&T 800 Plan K Service.'],
       ['NH-21-0008', '15-sep-21', 512, ..., '15-feb-22', 'Z', 0]],
      dtype=object)

Revisa el caso que investigaste en liga y replantea tu modelo
https://www.statology.org/logistic-regression-python/

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.values.reshape(22,11719), df.columns, test_size = 0.2, random_state = 21)

In [ ]:
pipe = make_pipeline(StandardScaler(), LogisticRegression())

In [ ]:
pipe.fit(x_train, y_train)

ValueError: ignored